In [7]:
import numpy as np
import matplotlib.pyplot as plt
from python_speech_features import mfcc

In [8]:
SNR = 17
np.random.seed(37)

data_type = 'raw'

time_len = 20 ##ms

if time_len==0.5:
    point_len = 25
else:
    point_len = int(45000*time_len/1000)
div_num = int(2250/point_len)
if div_num>10: div_num=10
    
label = []
for g1 in range(20):
    for g2 in range(20):
        dL = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        dL[g1] = 1
        dL[g2] = 1
        label.append(dL)

In [9]:
if data_type=='raw':
    time_len_= 20
    point_len_ = int(45000*time_len_/1000)
    
    trainX = np.zeros((20*19*200, point_len_))
    trainY = np.zeros((20*19*200, 20))
    count = 0
    for fq1 in range(20):
        for fq2 in range(20):
            if fq1==fq2:
                count += 1
                continue
            dY = label[20*fq1+fq2]
            freq1 = 600 + fq1*5
            freq2 = 600 + fq2*5
            t = np.linspace(0, time_len_/1000, point_len_)
            for n in range(200):
                phase1 = np.random.random()
                phase2 = np.random.random()

                y1 = np.sin((2*np.pi*freq1)*(t+phase1))
                y2 = np.sin((2*np.pi*freq2)*(t+phase2))

                wav = y1+y2
                noise = 2*np.random.random(len(wav))-1
                wav_rms = np.sqrt(np.mean(np.square(wav)))
                nos_rms = np.sqrt(np.mean(np.square(noise)))
                snr_ratio = wav_rms/nos_rms
                weight = snr_ratio/(10**(SNR/20))
                noise = weight*noise
                wav += noise

                trainX[(fq1*20+fq2)*200 + n - (count*200)] = wav
                trainY[(fq1*20+fq2)*200 + n - (count*200)] = dY

    testX = np.zeros((20*19*20, point_len_))
    testY = np.zeros((20*19*20, 20))
    count = 0
    for fq1 in range(20):
        for fq2 in range(20):
            if fq1==fq2:
                count += 1
                continue
            dY = label[20*fq1+fq2]
            freq1 = 600 + fq1*5
            freq2 = 600 + fq2*5
            t = np.linspace(0, time_len_/1000, point_len_)
            for n in range(20):
                phase1 = np.random.random()
                phase2 = np.random.random()

                y1 = np.sin((2*np.pi*freq1)*(t+phase1))
                y2 = np.sin((2*np.pi*freq2)*(t+phase2))

                wav = y1+y2
                noise = 2*np.random.random(len(wav))-1
                wav_rms = np.sqrt(np.mean(np.square(wav)))
                nos_rms = np.sqrt(np.mean(np.square(noise)))
                snr_ratio = wav_rms/nos_rms
                weight = snr_ratio/(10**(SNR/20))
                noise = weight*noise
                wav += noise

                testX[(fq1*20+fq2)*20 + n - (count*20)] = wav
                testY[(fq1*20+fq2)*20 + n - (count*20)] = dY
    
    trainX = trainX[:, :div_num*point_len]
    testX = testX[:, :div_num*point_len]
    
    trainX = trainX.reshape(-1, point_len)
    trainY_ = np.zeros((np.shape(trainX)[0], 20))
    testX = testX.reshape(-1, point_len)
    testY_ = np.zeros((np.shape(testX)[0], 20))
    
    for k in range(len(trainY)):
        trainY_[k*div_num:(k+1)*div_num] = trainY[k]
    
    for k in range(len(testY)):
        testY_[k*div_num:(k+1)*div_num] = testY[k]
    
    trainY = trainY_
    testY = testY_
    
    np.save('npy_data/raw/raw_%.1fms_trainX.npy'%time_len, trainX)
    np.save('npy_data/raw/raw_%.1fms_trainY.npy'%time_len, trainY)
    np.save('npy_data/raw/raw_%.1fms_testX.npy'%time_len, testX)
    np.save('npy_data/raw/raw_%.1fms_testY.npy'%time_len, testY)
    
    print('raw data shape  -  %.1f ms'%time_len)
    print('train set :', np.shape(trainX) , np.shape(trainY))
    print('test set :', np.shape(testX) ,np.shape(testY))

raw data shape  -  20.0 ms
train set : (76000, 900) (76000, 20)
test set : (7600, 900) (7600, 20)


In [4]:
if data_type=='fft':
    trainX = np.load('npy_data/raw/raw_%.1fms_trainX.npy'%time_len)
    trainY = np.load('npy_data/raw/raw_%.1fms_trainY.npy'%time_len)
    testX = np.load('npy_data/raw/raw_%.1fms_testX.npy'%time_len)
    testY = np.load('npy_data/raw/raw_%.1fms_testY.npy'%time_len)
    
    trainX = np.abs(np.fft.rfft(trainX)/len(trainX[0]))
    testX = np.abs(np.fft.rfft(testX)/len(testX[0]))
    
    np.save('npy_data/fft/fft_%.1fms_trainX.npy'%time_len, trainX)
    np.save('npy_data/fft/fft_%.1fms_trainY.npy'%time_len, trainY)
    np.save('npy_data/fft/fft_%.1fms_testX.npy'%time_len, testX)
    np.save('npy_data/fft/fft_%.1fms_testY.npy'%time_len, testY)
    
    print('fft data shape  -  %.1f ms'%time_len)
    print('train set :', np.shape(trainX) , np.shape(trainY))
    print('test set :', np.shape(testX) ,np.shape(testY))
    

In [6]:
if data_type=='mfcc':
    trainX = np.load('npy_data/raw/raw_%.1fms_trainX.npy'%time_len)
    trainY = np.load('npy_data/raw/raw_%.1fms_trainY.npy'%time_len)
    testX = np.load('npy_data/raw/raw_%.1fms_testX.npy'%time_len)
    testY = np.load('npy_data/raw/raw_%.1fms_testY.npy'%time_len)
    
    trainX_ = np.zeros((len(trainX),25))
    testX_ = np.zeros((len(testX),25))
    
    for k in range(len(trainX_)):
        mfwav = mfcc(trainX[k], 45000, winlen=point_len/45000, winstep=point_len/45000, 
                             numcep=25, nfilt = 40, nfft = point_len, lowfreq=20, preemph=0.97, 
                             ceplifter=20)
        trainX_[k] = mfwav[0]
    
    for k in range(len(testX_)):
        mfwav = mfcc(testX[k], 45000, winlen=point_len/45000, winstep=point_len/45000, 
                             numcep=25, nfilt = 40, nfft = point_len, lowfreq=20, preemph=0.97, 
                             ceplifter=20)
        testX_[k] = mfwav[0]
    
    trainX = trainX_
    testX = testX_
    
    np.save('npy_data/mfcc/mfcc_%.1fms_trainX.npy'%time_len, trainX)
    np.save('npy_data/mfcc/mfcc_%.1fms_trainY.npy'%time_len, trainY)
    np.save('npy_data/mfcc/mfcc_%.1fms_testX.npy'%time_len, testX)
    np.save('npy_data/mfcc/mfcc_%.1fms_testY.npy'%time_len, testY)
    
    print('mfcc data shape  -  %.1f ms'%time_len)
    print('train set :', np.shape(trainX) , np.shape(trainY))
    print('test set :', np.shape(testX) ,np.shape(testY))

mfcc data shape  -  0.5 ms
train set : (760000, 25) (760000, 20)
test set : (76000, 25) (76000, 20)


In [8]:
if data_type=='mem':
    point_len = int(np.ceil(time_len*5))
    div_num = int(250/point_len)
    if div_num>10: div_num=10
        
    trX = np.load('data/mem_2freq_trX.npy').reshape(-1, 6, 250)
    trY = np.load('data/mem_2freq_trY.npy')
    teX = np.load('data/mem_2freq_teX.npy').reshape(-1, 6, 250)
    teY = np.load('data/mem_2freq_teY.npy')
    
    trainX = trX[:,:,:div_num*point_len]
    trainY = trY
    testX = teX[:,:,:div_num*point_len]
    testY = teY 
    print(np.shape(trainX), np.shape(testX))
    
    num_list = np.arange(80000)
    for k in range(20):
        idx_tr = 400*k*10
        idx_te = 400*k
        trainX = np.delete(trainX, num_list[idx_tr:idx_tr+200], 0)
        trainY = np.delete(trainY, num_list[idx_tr:idx_tr+200], 0)
        testX = np.delete(testX, num_list[idx_te:idx_te+20], 0)
        testY = np.delete(testY, num_list[idx_te:idx_te+20], 0)
    
    trainX = trainX.transpose(0,2,1).reshape(-1, point_len, 6).transpose(0,2,1)
    trainY_ = np.zeros((np.shape(trainX)[0], 20))
    testX = testX.transpose(0,2,1).reshape(-1, point_len, 6).transpose(0,2,1)
    testY_ = np.zeros((np.shape(testX)[0], 20))
    
    for k in range(len(trainY)):
        trainY_[k*div_num:(k+1)*div_num] = trainY[k]
    
    for k in range(len(testY)):
        testY_[k*div_num:(k+1)*div_num] = testY[k]
    
    trainY = trainY_
    testY = testY_
    
    np.save('npy_data/mem/mem_%.1fms_trainX.npy'%time_len, trainX)
    np.save('npy_data/mem/mem_%.1fms_trainY.npy'%time_len, trainY)
    np.save('npy_data/mem/mem_%.1fms_testX.npy'%time_len, testX)
    np.save('npy_data/mem/mem_%.1fms_testY.npy'%time_len, testY)
    
    print('mem data shape  -  %.1f ms'%time_len)
    print('train set :', np.shape(trainX) , np.shape(trainY))
    print('test set :', np.shape(testX) ,np.shape(testY))
